<a href="https://colab.research.google.com/github/M-110/testing-with-pytest/blob/main/05_Plugins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Highlights
* Custom test status/headers
* Create plugin

Finding Plugins:

* https://docs.pytest.org/en/latest/plugins.html

* https://pypi.python.org

* https://github.com/pytest-dev

In [ ]:
!pip install pytest-cov

     |████████████████████████████████| 280 kB 8.3 MB/s 
     |████████████████████████████████| 242 kB 29.6 MB/s 
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: pytest
    Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4
  Attempting uninstall: coverage
    Found existing installation: coverage 3.7.1
    Uninstalling coverage-3.7.1:
      Successfully uninstalled coverage-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires coverage==3.7.1, but you have coverage 5.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
coveralls 0.5 requires coverage<3.999,>=3.6, but you have co

In [ ]:
!pip install git+https://github.com/pytest-dev/pytest-cov

  Cloning https://github.com/pytest-dev/pytest-cov to /tmp/pip-req-build-t6hinhb8
  Running command git clone -q https://github.com/pytest-dev/pytest-cov /tmp/pip-req-build-t6hinhb8


In [ ]:
!pip install pytest==6.2.4

In [ ]:
%%writefile conftest.py
def pytest_addoption(parser):
    """Turn nice features on with --nice option."""
    group = parser.getgroup('nice')
    group.addoption('--nice', action='store_true',
                    help='nice: turn failures into happy accident')


def pytest_report_header(config):
    """Thank the tester."""
    if config.getoption('--nice'):
        return "I'm so glad we'll be testing together today 😀."

def pytest_report_teststatus(report, config):
    nice = config.getoption('--nice')
    if report.when == 'call' and report.failed and nice:
        return report.outcome, 'A', 'HAPPY ACCIDENT 💡'
    elif report.when == 'call' and report.passed and nice:
        return report.outcome, 'P', 'PERFECTION 👍'

Writing conftest.py


In [ ]:
%%writefile my_test.py
def test_this():
  assert 1 == 0

def test_That():
  assert 1 == 1

Writing my_test.py


In [ ]:
!python3 -m pytest my_test.py --nice -v

============================= test session starts ==============================
platform linux -- Python 3.7.11, pytest-6.2.4, py-1.10.0, pluggy-0.13.1 -- /usr/bin/python3
cachedir: .pytest_cache
I'm so glad we'll be testing together today 😀.
rootdir: /content
plugins: cov-2.12.1, typeguard-2.7.1
collected 2 items                                                              

my_test.py::test_this HAPPY ACCIDENT 💡                                  [ 50%]
my_test.py::test_That PERFECTION 👍                                      [100%]

=================================== FAILURES ===================================
__________________________________ test_this ___________________________________

    def test_this():
>     assert 1 == 0
E     assert 1 == 0
E       +1
E       -0

my_test.py:2: AssertionError
=========================== short test summary info ============================
HAPPY ACCIDENT 💡 my_test.py::test_this - assert 1 == 0
========================= 1 failed, 1 passed in 0

# Create installable plugin

In [ ]:
%%bash
mkdir pytest-nice
cd pytest-nice
touch LICENSE
touch README.rst
touch pytest_nice.py
touch setup.py
mkdir tests
cd tests
touch conftest.py
touch test_nice.py

In [ ]:
%%writefile pytest-nice/pytest_nice.py
import pytest

def pytest_addoption(parser):
  group = parser.getgroup('nice')
  group.addoption('--nice', action='store_true',
                  help='nice: turn FIALED into OPPORTUNITY for improvement')

def pytest_report_header(config):
  if config.getoption('nice'):
    return 'Thanks for running the tests!'
  
def pytest_report_teststatus(config, report):
  if report.when == 'call':
    if report.failed and config.getoption('nice'):
      return report.outcome, 'O', 'OPPORTUNITY for improvement'

Overwriting pytest-nice/pytest_nice.py


In [ ]:
%%writefile pytest-nice/setup.py
from setuptools import setup

setup(name='pytest-nice',
      version='0.1.0',
      description='A nice plugin',
      uirl='https://colab.research.google.com',
      author='Nature',
      author_email='the_wind_at_your_back@nature.earth',
      license='free',
      py_modules=['pytest_nice'],
      install_requires=['pytest'],
      entry_points={'pytest11': ['nice=pytest_nice',],})

Overwriting pytest-nice/setup.py


In [ ]:
%%writefile pytest-nice/tests/conftest.py
pytest_plugins = 'pytester'

Overwriting pytest-nice/tests/conftest.py


In [ ]:
%%writefile pytest-nice/tests/test_nice.py
import pytest


def test_pass_fail(testdir):
  # create a temporary pytest test module
  testdir.makepyfile("""
  def test_pass():
    assert 1 == 1

  def test_fail():
    assert 1 == 2

  """)
  result = testdir.runpytest()

  result.stdout.fnmatch_lines(['*.F*'])

  assert result.ret == 1

@pytest.fixture()
def sample_test(testdir):
  testdir.makepyfile("""
  def test_pass():
    assert 1 == 1
  
  def test_fail():
    assert 1 == 2
    """)
  return testdir

def test_with_nice(sample_test):
  result = sample_test.runpytest('--nice')
  result.stdout.fnmatch_lines(['*.O*'])
  assert result.ret == 1

def test_with_nice_verbose(sample_test):
  result = sample_test.runpytest('-v', '--nice')
  result.stdout.fnmatch_lines(['*::test_fail OPPORTUNITY for improvement*'])
  assert result.ret == 1

def test_not_nice_verbose(sample_test):
  result = sample_test.runpytest('-v')
  result.stdout.fnmatch_lines(['*::test_fail FAILED*'])
  assert result.ret == 1

def test_header(sample_test):
  result = sample_test.runpytest('--nice')
  result.stdout.fnmatch_lines(['*Thanks for running the tests*'])

def test_header_not_nice(sample_test):
  result =  sample_test.runpytest()
  thanks_message = 'Thanks for running the tests!'
  assert thanks_message not in result.stdout.str()

def test_help_message(testdir):
  result = testdir.runpytest('--help')

  result.stdout.fnmatch_lines(['nice:', '*nice: turn FIALED into OPPORTUNITY for improvement*'])

Overwriting pytest-nice/tests/test_nice.py


In [ ]:
%%bash
cd pytest-nice
pip install .

Processing /content/pytest-nice
  Created wheel for pytest-nice: filename=pytest_nice-0.1.0-py3-none-any.whl size=1947 sha256=938fee4cd5c9e5849ffbe6b13202c26db1374d3b3d6c4d23b849c834d247e932
  Stored in directory: /root/.cache/pip/wheels/2c/9c/ab/a858f36277f84110f985da5319ef9011e13b359f9fdff40ec0
Successfully built pytest-nice


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [ ]:
!python3 -m pytest pytest-nice -v

Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/pytest/__main__.py", line 5, in <module>
    raise SystemExit(pytest.console_main())
  File "/usr/local/lib/python3.7/dist-packages/_pytest/config/__init__.py", line 185, in console_main
    code = main()
  File "/usr/local/lib/python3.7/dist-packages/_pytest/config/__init__.py", line 143, in main
    config = _prepareconfig(args, plugins)
  File "/usr/local/lib/python3.7/dist-packages/_pytest/config/__init__.py", line 319, in _prepareconfig
    pluginmanager=pluginmanager, args=args
  File "/usr/local/lib/python3.7/dist-packages/pluggy/hooks.py", line 286, in __call__
    return self._hookexec(self, self.get_hookimpls(), kwargs)
  File "/usr/local/lib/python3.7/dist-packages/pluggy/manager.py", line 93, in _hookexec
    re

In [ ]:
!pip uninstall pytest-nice